In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

In [2]:
labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)

51

In [3]:
#k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

In [4]:
config = {
        'df_path': '../input/user1.features_labels.csv',
        #'labels': labels
}
    
har = utils.HAR(config)
har.run()
#har.hypertunning()

2022-03-21 18:20:12.216446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 18:20:12.248787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-21 18:20:12.248802: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-21 18:20:12.249459: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Epoch 1/40
162/162 [==============================] - 1s 2ms/step - loss: 0.7598 - categorical_accuracy: 0.5407 - val_loss: 0.7405 - val_categorical_accuracy: 0.5509
Epoch 2/40
162/162 [==============================] - 0s 930us/step - loss: 0.7296 - categorical_accuracy: 0.5829 - val_loss: 0.7313 - val_categorical_accuracy: 0.5509
Epoch 3/40
162/162 [==============================] - 0s 904us/step - loss: 0.7214 - categorical_accuracy: 0.5829 - val_loss: 0.7276 - val_categorical_accuracy: 0.5509
Epoch 4/40
162/162 [==============================] - 0s 845us/step - loss: 0.7172 - categorical_accuracy: 0.5829 - val_loss: 0.7255 - val_categorical_accuracy: 0.5509
Epoch 5/40
162/162 [==============================] - 0s 826us/step - loss: 0.7145 - categorical_accuracy: 0.5829 - val_loss: 0.7239 - val_categorical_accuracy: 0.5509
Epoch 6/40
162/162 [==============================] - 0s 864us/step - loss: 0.7126 - categorical_accuracy: 0.5909 - val_loss: 0.7227 - val_categorical_accuracy: 0

In [5]:
def nan_mse(y_actual, y_predicted):
    stack = tf.stack((tf.math.is_nan(y_actual), 
                      tf.math.is_nan(y_predicted)),
                     axis=1)
    is_nans = tf.keras.backend.any(stack, axis=1)
    per_instance = tf.where(is_nans,
                            tf.zeros_like(y_actual),
                            tf.square(tf.subtract(y_predicted, y_actual)))
    print(stack)
    print(per_instance)
    return tf.reduce_mean(per_instance, axis=0)

print(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))

tf.Tensor(
[[False False]
 [False False]
 [ True False]
 [False False]
 [False  True]], shape=(5, 2), dtype=bool)
tf.Tensor([0. 0. 0. 1. 0.], shape=(5,), dtype=float32)
tf.Tensor(0.2, shape=(), dtype=float32)


In [6]:
def remove_nans(x, y):
    #mask = ~np.isnan(x) & ~np.isnan(y)
    mask = tf.math.logical_and(tf.math.logical_not(tf.math.is_nan(x)), tf.math.logical_not(tf.math.is_nan(y)))
    
    marra = tf.boolean_mask(x, mask) #np.ma.MaskedArray(a, mask=~mask)
    marrb = tf.boolean_mask(y, mask) #np.ma.MaskedArray(b, mask=~mask)

    return marra, marrb #np.ma.compressed(marra), np.ma.compressed(marrb)


def nan_bce(y_actual, y_predicted):
    bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    y_predicted_masked, y_actual_masked = remove_nans(y_predicted, y_actual)
    
    return bce(y_predicted_masked, y_actual_masked)

print(nan_bce([1.,1.,np.nan,1.,0.], [-18.6, 0.51, 2.94, -12.8,np.nan]))
#print(nan_bce([1.,1.,0,1.,0.], [-18.6, 0.51, 2.94, -12.8,0]))

tf.Tensor(11.609928, shape=(), dtype=float32)


In [7]:
x = raw[raw.columns.drop(raw.filter(regex='label:'))]
y = raw.filter(regex='label:')

NameError: name 'raw' is not defined

In [ ]:
x

In [ ]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce([1],[18.6]).numpy()

In [ ]:
a, b = [1.,1.,np.nan,1.,0.], [-18.6, 0.51, 2.94, -12.8,np.nan]
mask = ~np.isnan(a) & ~np.isnan(b)
print(mask)
marr = np.ma.MaskedArray(a, mask=~mask)
marr
np.ma.compressed(marr)
#np.ma.compress_rows(marr)
#mask = mask.astype(int)
#a[mask]
#a = a[mask]
#b = b[mask]

In [ ]:
nan_bce(a, b)

In [ ]:
nan_bce([1.,1.,1.], [-18.6, 0.51, -12.8])

In [ ]:
remove_nans(a, b)

In [ ]:
aa, bb = remove_nans(a,b)
bce(aa, bb)

In [ ]:
x = tf.constant([5.0, np.nan, 6.8, np.nan, np.inf])
 #==> [False, True, False, True, False]
tf.math.logical_not(tf.math.is_nan(x))    

In [ ]:
tf.math.logical_and(tf.math.logical_not(tf.math.is_nan(x)), tf.math.is_nan(x))

In [ ]:
y_pred = har.mlp.model.predict(har.data.x_test)
y_test = har.data.y_test

In [ ]:
from sklearn.metrics import classification_report

def avg_multilabel_BA(y_truei, y_predi):
    ba_array = []
    y_true, y_pred = remove_nans_np(y_truei, y_predi)
    print(y_true.shape)
    print(y_truei.shape)
    #print(len(y_true))
    #print(len(y_truei))
    for i in range(y_pred.shape[1]):
        try:
            report = classification_report(y_true.to_numpy()[:, i], (y_pred[:, i] > 0.5), output_dict=True, zero_division=0)
        except:
            report = classification_report(y_true[:, i], (y_pred[:, i] > 0.5), output_dict=True, zero_division=0)
        #sensitivity = report['1.0']['recall'] # tp / (tp + fn)
        specificity = report['0.0']['recall'] #specificity = tn / (tn+fp)
        try:
            sensitivity = report['1.0']['recall'] # tp / (tp + fn)
        except:
            sensitivity = specificity # tp / (tp + fn)
        ba = 0.5*(specificity+sensitivity)
        ba_array.append(ba)
    return np.mean(ba_array)


def remove_nans_np(x, y):
    mask = ~np.isnan(x) & ~np.isnan(y)
    #mask = math.logical_and(math.logical_not(math.is_nan(x)), math.logical_not(math.is_nan(y)))
    
    marra = np.ma.MaskedArray(x, mask=~mask)
    marrb = np.ma.MaskedArray(y, mask=~mask)
    print(marrb.shape)
    #print(marrb)

    return np.ma.compressed(marra).reshape(-1, 4), np.ma.compressed(marrb).reshape(-1, 4)

In [ ]:
avg_multilabel_BA(y_test, y_pred)